### Binary Features: Flag, Bool, True - False
* Bu konuda genel kapsayıcı ve kabul gören net bir literatür yoktur. Çünkü özellik çıkarımı işi problemden probleme değişebilir. Özelinde yöntemden yönteme değişebilir. Burada karşımıza gelebilecek aday isimlendirmelerini (Flag, Bool, True - False) bu sebeple birlikte verilmiştir. Burada yapacak olduğumuz işlem 1 0 şeklinde var olan değişkenler üzerinden yeni değişkenler türetmektir. 
* DİKKAT: Daha önce label encoding ve binary encoding yapmıştık. Bu o değil. Var olan değişkenşn içinden böyle değişkenler türetmek yani yeni bir şeyler türetmek ile ilgileniyoruz, var olanları değiştirmekle değil.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data

In [3]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


* DİKKAT: Titanic veri setinde cabin değişkeni vardı. Burada bazı değerlerde NA bazı değerlerde de NA olmama durumu vardı. Şimdi öyle bir işlem yapmak istiyoruz ki NA gördüğümüz yerlere 1 NA olmayan yerlere 0 yazmak istiyoruz. Normalde bu cabin değişkeni çöp bir değişken çünkü çok fazla sayıda eksik değer var. Ama şunu merak ediyoruz:Bu kabinlere NA olanlara 1 NA olmayanlara 0 yazıp bir gözlemleyelim. Gerçekten anlamsız bir değişken mi, buna bakmak istiyoruz. Silmeli miyiz sorusunu araştırıyoruz.

In [5]:
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype('int')
df.head()

# Şu anda bizim için anlamsız olan değişken daha dikkate değer hale geldi ama hala emin değiliz.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0


In [6]:
# Bağımlı değişkenimiz Survived'a göre ortalama alıp yeni oluşan değişkene göre groupby'a alalım.
df.groupby("NEW_CABIN_BOOL").agg({"Survived": "mean"})

# DİKKAT: Kabini dolu olanların hayatta kalma olasılığı kabini dolu olmayanlara göre daha fazla.
# Daha öncesinde çöp gibi gözüken bu değişken en kritik olabilecek değişkenlerden birisi haline geldi.

,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


* Peki bu kayda değer bir ilişki midir?
* DİKKAT: Kabin numarası 1 olan herkesi topladık ve hayatta kalıp kalmama durumunu görebilmek için ortalamasını aldık. Dolu kabinlerin hayatta kalması çok yüksek, boş olanların neden düşük? Şimdi bununla ilgilenmiyoruz ama şunu mmerak ediypruz: Ya acaba gerçekten bu bir ilişki taşıyor olabilir mi? 
* Buraya istatistiki bir test/oran testi yapalım. İki grubun oranını kıyaslama testi yapalım:

In [8]:
# !pip install statsmodels

  Using cached statsmodels-0.13.2-cp39-cp39-win_amd64.whl (9.1 MB)
  Using cached patsy-0.5.2-py2.py3-none-any.whl (233 kB)


In [9]:
from statsmodels.stats.proportion import proportions_ztest
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
                                            df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum()  --> Kabin numarası olan hayatta olan kaç kişi var
# df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()  --> Kabin numarası olmayan...
# Oran testini yapabilmek için gerekli olan 2 parametre var birisi count birisi nobs(gözlem sayısı)

Test Stat = 9.4597, p-value = 0.0000


* DİKKAT: Oldukça önemli bir noktadayız. Proportion z testinin hipotezi  şu şekildedir: p1 ve p2 oranları arasında fark yoktur der. p1 ve p2 oranları iki grubun hayatta kalıp kalmama durumunu ifade ediyor. İkisi arasında fark yoktur diyen H0 Hipotezi p-value değeri 0.05' ten küçük olduğundan dolayı reddedilir. Yani aralarında istatistiki olarak anlamlı bir farklılık var gibi gözüküyor.
* İKİ KERE DİKKAT: Çok değişkenli etkiyi bilmiyoruz. İki tane değişkenin sanki sadece ikisi birlikte oluşmuş gibi değerlendirerek inceledik. Ama bu yapılar tek başına oluşmadı ki. Yani Survived değişkeninin ortaya çıkışı burada ele aldığımız new_cabin_bool değişkeni ile ortaya çıkmadı ki ama çıkmış olabilir. Şu anki kanaatimiz de çıkmış olabileceğine yönelik olsa da hala bilemiyoruz. Dolayısıyla çok değişkenli etkiyi bbilmiyoruz. Bu değişkenleri birlikte modele soktuğumuzda bu çok değişkenli etkiyi gözlemleme imkanımız olacak. O durumda bu değişkenin (new_cabin_bool ) anlamlı olup olmadığını daha iyi değerlendiriyor olacağız. Ama şu anda ilerlemek ya da bu değişkeni kabul edip etmemek için fikir edinmeye çalıştığımızda yeterli ispatı/delili bulduk. Bu değşken artık cepte. Ama tekrar söylüyoruz çok değişkenli etkiyi bilmiyoruz dolayısıyla bu bizimiçin çok önemli bir değişkendir genellemesini yapamıyoruz. Göz önünde bulunduruyoruz sadece, kayda değerdir diyoruz.
* Bu veri setinde SibSp ve parch adında iki değişkenimiz var. Bunlar yakın ve uzak olabilecek akrabalıkları ifade ediyor. Yani kişinin gemideki akrabalıklarını ifade eden iki tane değişken var. Bunların toplamı eğer sıfırdan büyükse yeni bir değişken oluştur diyoruz. Demek ki bu kişi yalnız değilmiş (NO) şeklinde NEW_IS_ALONE değişkenine yeni bir değer ata diyoruz. 
* DİKKAT: Normalde iki değişken var. ikisini bir araya getirdik, bir de koşul koyduk. Bu koşula göre yeni bir değişken oluşturduk. Eğer sıfırdan büyükse yalnız değil, sıfır ise yalnız diyoruz. Olası featureları türetmeye çalışıyoruz. Yorumumuz belki kişi gemide tek olup olmamasına göre hayatta kalma refleksleri şekillenmiştir şeklinde oluyor. Annesi babası çocukları ya da daha uzaktan akrabaları olmama durumu onun gemideki ya da hayatta kalma sğrecindeki çabasını etkilemiş olabilir. Neticesinde bir sonuç çıkmayabilir de, değişken üretmenin güzel noktası olası tüm özellikleri üretmeye çalışmaktır. 

In [11]:
# Akrabalıkları içeren değişkenle yapalım aynı işlemleri:
df.loc[((df['SibSp'] + df['Parch']) > 0), "NEW_IS_ALONE"] = "NO"

In [12]:
df.loc[((df['SibSp'] + df['Parch']) == 0), "NEW_IS_ALONE"] = "YES"

In [13]:
df.groupby("NEW_IS_ALONE").agg({"Survived": "mean"})

# Aralarında bir fark var gibi gözüküyor: 
# Bu etki şans eseri ortaya çıkmış olabilir, çok değişkenli başka etkiler söz konusu olabilir.
# Sadece ucundan fikir edinmek için baktığımızda fark var gibi gözüküyor.

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


In [14]:
# Hipotez testi yapalımi proportional testimizi buraya da getiriyoruz:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].sum(),
                                             df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],
                                            df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].shape[0]])


In [15]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -6.0704, p-value = 0.0000


In [17]:
# Anlamlı bir farklılık var H0 Hipotezi reddedilir. Modelleme bölümünde bu etki daha net gözüküyor olacaktır.